In [1]:
import re
import glob
import xlwings as xw
import math
from pathlib import Path
import random
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as mpl
from matplotlib import cm
import matplotlib.pyplot as plt
import matplotlib.colors as mclr
import plotly.express as px
import statsmodels.api as sm
import pylab as py
import os, cv2, glob, tempfile
import joblib

from scipy import stats
from scipy.stats import pearsonr

import sklearn
from sklearn import datasets
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import minmax_scale
from sklearn.preprocessing import MaxAbsScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import QuantileTransformer
from sklearn.datasets import make_blobs

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import VotingClassifier
from sklearn import metrics
from sklearn.metrics import recall_score, precision_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import matthews_corrcoef
make_scorer = sklearn.metrics.make_scorer
f1 = make_scorer(f1_score, pos_label=1, average="binary")
from sklearn.metrics import classification_report
from sklearn.inspection import permutation_importance
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate, KFold
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.model_selection import RepeatedStratifiedKFold

import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import BatchNormalization, Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense
from keras.optimizers import Adam
from keras import utils as np_utils
from keras import models
import keras_tuner as kt
from scipy import signal

import config as config
print(config.__path__)
from keras.callbacks import EarlyStopping
from keras.utils import to_categorical
from keras import backend as K

import argparse
# from sklearn.utils import class_weight
print(tf.__version__)
print(keras.__version__)

import shutil
import xlwings as xw

%matplotlib inline
mpl.use("Agg")

['C:\\Users\\user\\anaconda3\\lib\\site-packages\\config']
2.12.0-dev20221107
2.12.0


In [2]:
## Build XGBoost Model
from copy import deepcopy
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.graphics.tsaplots import plot_acf,plot_pacf
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV,train_test_split
from skopt import BayesSearchCV
from sklearn.metrics import accuracy_score,f1_score,roc_auc_score,confusion_matrix,roc_curve
import xgboost as xgb
import time
import warnings

# 1. Data Preparation

In [3]:
trainDEFSDf = pd.read_csv(r"I:\\3_output_raMSIn\\3_3_Output_raMSIn_HKU_Ingested4ALL\\10ppm\\df_train_dbMSIn10ppm4nonInDI_STDnorm.csv")
diDEFSDf = pd.read_csv(r"I:\\3_output_raMSIn\\3_3_Output_raMSIn_HKU_Ingested4ALL\\10ppm\\df_nonInDI_dbMSIn10ppm4nonInDI_STDnorm.csv")

In [4]:
trainDEFSDf

,pixel_id,311.1684,269.2486,215.0328,277.2172,435.2965,309.1704,738.5066,241.2173,250.1449,...,265.1478,794.5338,883.533,227.2016,233.1546,124.0072,602.4456,514.2846,867.5147,type
0,HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slid...,3.687726,0.568057,-1.174267,-0.396204,0.529319,4.502233,-1.001765,1.063677,1.380095,...,4.516408,-0.453354,-0.488411,0.707264,0.763448,-0.085210,-0.117664,-0.425952,-0.442700,0
1,HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slid...,5.100212,1.496236,-0.377348,-0.337222,0.260481,5.620431,-1.120449,1.372003,3.286912,...,5.492450,-0.453354,-0.288439,0.854702,1.669974,1.225312,-0.117664,-0.425952,-0.442700,0
2,HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slid...,3.445872,0.628390,-0.825027,-0.338485,-0.133034,3.713885,-1.053360,0.988516,2.284046,...,4.409207,-0.453354,-0.472931,0.999037,1.926641,-0.954524,-0.117664,-0.425952,-0.442700,0
3,HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slid...,3.196718,0.434454,-0.418413,-0.688982,-0.082019,3.502578,-1.000024,1.280699,2.047447,...,4.249314,1.738526,0.592190,0.580417,1.504607,-1.007959,-0.117664,-0.425952,1.825934,0
4,HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slid...,1.997115,0.757400,-1.300444,-0.716567,0.627675,2.567365,-1.021176,0.782820,1.036886,...,2.983452,-0.453354,-0.367406,0.733112,0.493555,-0.243213,-0.117664,-0.425952,-0.442700,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90955,HKULiver_Post29wk_HCCLiver_Left_Mice12_Slide4F...,0.410525,-1.038074,0.041030,-0.878750,1.088504,0.737200,-1.400189,-1.168553,0.417574,...,0.504216,-0.453354,-0.926756,0.483292,0.701020,0.931225,-0.117664,2.184668,-0.442700,1
90956,HKULiver_Post29wk_HCCLiver_Left_Mice12_Slide4F...,0.252412,-1.038074,-0.899504,-0.994299,1.347635,0.869989,-1.340002,0.716735,2.304066,...,0.259997,-0.453354,-0.427607,0.487766,0.443162,-0.259778,-0.117664,-0.425952,-0.442700,1
90957,HKULiver_Post29wk_HCCLiver_Left_Mice12_Slide4F...,0.357466,1.260345,-0.920750,-0.918819,0.877777,0.521725,-0.798359,-1.168553,1.567573,...,-0.735005,-0.453354,-0.606650,0.618718,1.246264,0.181284,-0.117664,-0.425952,2.018798,1
90958,HKULiver_Post29wk_HCCLiver_Left_Mice12_Slide4F...,0.526576,-1.038074,-1.209441,-0.506793,0.670780,0.702305,-0.967547,-1.168553,2.067876,...,0.671902,-0.453354,-1.055427,0.576152,0.669682,-0.230168,-0.117664,-0.425952,-0.442700,1


In [10]:
can = []
for i in list(range(0,4)) + list(range(5,8)) + list(range(9,10)) + list(range(15,17)) + list(range(19,23)) + [len(list(trainDEFSDf.columns))-1]:
    can.append(list(trainDEFSDf.columns)[i])

dican = []
for i in list(range(0,4)) + list(range(5,8)) + list(range(9,10)) + list(range(15,17)) + list(range(19,23)) + [len(list(diDEFSDf.columns))-1]:
    dican.append(list(diDEFSDf.columns)[i])

In [11]:
can

['pixel_id',
 '311.1684',
 '269.2486',
 '215.0328',
 '435.2965',
 '309.1704',
 '738.5066',
 '250.1449',
 '794.5338',
 '883.533',
 '124.0072',
 '602.4456',
 '514.2846',
 '867.5147',
 'type']

In [12]:
dican

['pixel_id',
 '311.1684',
 '269.2486',
 '215.0328',
 '435.2965',
 '309.1704',
 '738.5066',
 '250.1449',
 '794.5338',
 '883.533',
 '124.0072',
 '602.4456',
 '514.2846',
 '867.5147',
 'type']

In [14]:
## input training set ## 90960 x 20 df
trainDEFSDf = pd.read_csv(r"I:\\3_output_raMSIn\\3_3_Output_raMSIn_HKU_Ingested4ALL\\10ppm\\df_train_dbMSIn10ppm4nonInDI_STDnorm.csv")
#trainDEFSDf[trainDEFSDf.type .== 1, :]
## calculate weight ## 0: 47449, 1: 43511
trainDEFSDf = trainDEFSDf[can]
Yy_train = trainDEFSDf["type"]  # 0.9585; 1.0453
sampleW = []
for w in Yy_train:
    if w == 0:
        sampleW.append(0.9585)
    elif w == 1:
        sampleW.append(1.0453) 


In [15]:
## input ext val set ## 6075 x 20 df
extDEFSDf = pd.read_csv(r"I:\\3_output_raMSIn\\3_3_Output_raMSIn_HKU_Ingested4ALL\\10ppm\\df_ext_dbMSIn10ppm4nonInDI_STDnorm.csv")
#extDEFSDf[extDEFSDf.type .== 1, :]
## calculate weight ## 0: 2943, 1: 3132
extDEFSDf = extDEFSDf[can]
Yy_ext = extDEFSDf["type"]  # 1.0321; 0.9698
sampleExtW = []
for w in Yy_ext:
    if w == 0:
        sampleExtW.append(1.0321)
    elif w == 1:
        sampleExtW.append(0.9698) 


In [16]:
## input ingested set ## 97035 x 20 df
ingestedDEFSDf = pd.read_csv(r"I:\\3_output_raMSIn\\3_3_Output_raMSIn_HKU_Ingested4ALL\\10ppm\\df_ingested_dbMSIn10ppm4nonInDI_STDnorm.csv")
#ingestedDEFSDf[extDEFSDf.type .== 1, :]
## calculate weight ## 0: 50392, 1: 46643
ingestedDEFSDf = ingestedDEFSDf[can]
Yy_ingested = ingestedDEFSDf["type"]  # 0.9628; 1.0402
sampleIngestedW = []
for w in Yy_ingested:
    if w == 0:
        sampleIngestedW.append(0.9628)
    elif w == 1:
        sampleIngestedW.append(1.0402)


In [17]:
## input FNA set ## 88701 x 20 df
fnaDEFSDf = pd.read_csv(r"I:\\3_output_raMSIn\\3_3_Output_raMSIn_HKU_Ingested4ALL\\10ppm\\df_FNA_dbMSIn10ppm4nonInDI_STDnorm.csv")
#fnaDEFSDf[fnaDEFSDf.type .== 1, :]
## calculate weight ##  0: 44540, 1: 44161
fnaDEFSDf = fnaDEFSDf[can]
Yy_FNA = fnaDEFSDf["type"]  # 0.9957; 1.0043
sampleFNAW = []
for w in Yy_FNA:
    if w == 0:
        sampleFNAW.append(0.9957)
    elif w == 1:
        sampleFNAW.append(1.0043)


In [18]:
## input DirectIn set ## 88701 x 20 df
diDEFSDf = pd.read_csv(r"I:\\3_output_raMSIn\\3_3_Output_raMSIn_HKU_Ingested4ALL\\10ppm\\df_nonInDI_dbMSIn10ppm4nonInDI_STDnorm.csv")
#diDEFSDf[diDEFSDf.type .== 1, :]
## calculate weight ## 0: 3027, 1: 3030
diDEFSDf = diDEFSDf[dican]

Yy_DI = diDEFSDf["type"]  # 1.0007; 0.9993
sampleDiW = []
for w in Yy_DI:
    if w == 0:
        sampleDiW.append(1.0007)
    elif w == 1:
        sampleDiW.append(0.9993)


In [19]:
diDEFSDf

,pixel_id,311.1684,269.2486,215.0328,435.2965,309.1704,738.5066,250.1449,794.5338,883.533,124.0072,602.4456,514.2846,867.5147,type
0,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_...,0.402342,0.913244,0.790740,0.518836,1.697873,0.601978,2.971000,0.759606,0.488453,-0.440759,-0.480828,0.532073,-0.178070,0
1,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_...,0.314493,1.539995,-0.020203,0.742545,1.609584,0.390074,2.805791,-0.556675,0.246011,-0.067056,-0.480828,0.443193,-0.795131,0
2,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_...,0.841655,1.554217,0.631070,0.796962,2.182584,1.755656,3.286110,1.184002,0.950243,0.193754,-0.480828,0.989125,0.292370,0
3,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_...,0.621964,2.090087,0.242283,0.596528,1.903016,0.849887,3.016573,-0.313031,1.079633,0.218992,-0.480828,0.801811,-0.101963,0
4,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_...,0.303707,1.234845,0.129289,0.913738,1.677419,0.944668,3.116293,-0.299362,0.548905,0.047736,-0.480828,0.783867,-0.033724,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3023,HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S1...,0.046377,0.004282,1.328597,0.529506,-0.172416,0.489131,-0.437109,0.114391,1.053762,0.593882,3.261784,1.369131,0.417708,1
3024,HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S1...,-1.799346,-1.043795,-1.052138,-1.475923,-1.114427,1.334575,0.064193,0.762855,0.599189,-0.598447,1.322839,0.404445,0.710634,1
3025,HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S1...,-0.519249,-1.667011,1.227576,0.768313,-0.360029,0.180897,-0.437109,0.074299,0.109578,1.311032,-0.480828,1.725927,-0.031510,1
3026,HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S1...,-0.419602,-0.812576,1.820478,-0.280928,-0.357697,0.403818,-0.437109,0.670855,0.995031,-0.133601,2.180190,1.301959,0.232500,1


In [20]:
## define functions for performace evaluation ##

# Average score
def avgScore(arrAcc, cv):
    sumAcc = 0
    for acc in arrAcc:
        sumAcc += acc
    return sumAcc / cv


In [21]:
trainDEFSDf[trainDEFSDf.columns[1:-1]]

,311.1684,269.2486,215.0328,435.2965,309.1704,738.5066,250.1449,794.5338,883.533,124.0072,602.4456,514.2846,867.5147
0,3.687726,0.568057,-1.174267,0.529319,4.502233,-1.001765,1.380095,-0.453354,-0.488411,-0.085210,-0.117664,-0.425952,-0.442700
1,5.100212,1.496236,-0.377348,0.260481,5.620431,-1.120449,3.286912,-0.453354,-0.288439,1.225312,-0.117664,-0.425952,-0.442700
2,3.445872,0.628390,-0.825027,-0.133034,3.713885,-1.053360,2.284046,-0.453354,-0.472931,-0.954524,-0.117664,-0.425952,-0.442700
3,3.196718,0.434454,-0.418413,-0.082019,3.502578,-1.000024,2.047447,1.738526,0.592190,-1.007959,-0.117664,-0.425952,1.825934
4,1.997115,0.757400,-1.300444,0.627675,2.567365,-1.021176,1.036886,-0.453354,-0.367406,-0.243213,-0.117664,-0.425952,-0.442700
...,...,...,...,...,...,...,...,...,...,...,...,...,...
90955,0.410525,-1.038074,0.041030,1.088504,0.737200,-1.400189,0.417574,-0.453354,-0.926756,0.931225,-0.117664,2.184668,-0.442700
90956,0.252412,-1.038074,-0.899504,1.347635,0.869989,-1.340002,2.304066,-0.453354,-0.427607,-0.259778,-0.117664,-0.425952,-0.442700
90957,0.357466,1.260345,-0.920750,0.877777,0.521725,-0.798359,1.567573,-0.453354,-0.606650,0.181284,-0.117664,-0.425952,2.018798
90958,0.526576,-1.038074,-1.209441,0.670780,0.702305,-0.967547,2.067876,-0.453354,-1.055427,-0.230168,-0.117664,-0.425952,-0.442700


In [22]:
import warnings
warnings.filterwarnings('ignore')

# 2. Unsupervised Learning

## 2.1. Dataset Preparation

In [23]:
from matplotlib.pyplot import figure
plt.rcParams['figure.figsize'] = [10, 10]
plt.grid(visible=None)
plt.close()
figure(figsize=(10, 10))

<Figure size 720x720 with 0 Axes>

In [24]:
df_ROI_for_ML_Opti = pd.concat([trainDEFSDf, extDEFSDf, fnaDEFSDf, diDEFSDf]).set_index('pixel_id')

In [25]:
df_ROI_for_ML_Opti_ingested = ingestedDEFSDf.set_index('pixel_id')

In [26]:
df_ROI_for_ML_Opti_train = trainDEFSDf.set_index('pixel_id')

In [27]:
df_ROI_for_ML_Opti_ext = extDEFSDf.set_index('pixel_id')

In [28]:
df_ROI_for_ML_Opti_FNA = fnaDEFSDf.set_index('pixel_id')

In [29]:
df_ROI_for_ML_Opti_di = diDEFSDf.set_index('pixel_id')

In [30]:
df_ROI_for_ML_Opti_di.describe()

,311.1684,269.2486,215.0328,435.2965,309.1704,738.5066,250.1449,794.5338,883.533,124.0072,602.4456,514.2846,867.5147,type
count,3.028000e+03,3.028000e+03,3.028000e+03,3.028000e+03,3.028000e+03,3.028000e+03,3.028000e+03,3.028000e+03,3.028000e+03,3.028000e+03,3.028000e+03,3.028000e+03,3.028000e+03,3028.000000
mean,3.556527e-17,3.668356e-17,-1.899259e-17,4.651981e-16,-5.811438e-17,-5.695301e-16,1.401393e-14,-3.534528e-17,4.950355e-16,7.631867e-17,9.719621e-15,5.951499e-16,8.252517e-16,0.500330
std,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.500082
min,-2.776735e+00,-3.684381e+00,-2.682728e+00,-1.475923e+00,-1.701767e+00,-2.694045e+00,-4.371089e-01,-3.178069e+00,-2.324191e+00,-3.259495e+00,-4.808281e-01,-2.716400e+00,-2.139572e+00,0.000000
25%,-7.685900e-01,-6.134711e-01,-6.067875e-01,-5.787979e-01,-6.450113e-01,-6.052840e-01,-4.371089e-01,-4.790477e-01,-4.527378e-01,-3.936825e-01,-4.808281e-01,-7.348058e-01,-5.309773e-01,0.000000
50%,9.775581e-02,-2.030662e-01,9.546162e-03,1.457927e-01,-3.576545e-01,-1.598415e-01,-4.371089e-01,6.858819e-02,8.349740e-02,1.481650e-01,-4.808281e-01,-4.128838e-01,2.789193e-02,1.000000
75%,7.284643e-01,3.619300e-01,6.133208e-01,7.469444e-01,2.576578e-01,3.912102e-01,1.578378e-03,5.934633e-01,5.960008e-01,6.713227e-01,-4.808281e-01,7.484676e-01,5.615214e-01,1.000000
max,5.686665e+00,4.275386e+00,4.499246e+00,3.235670e+00,3.719943e+00,6.905744e+00,4.629420e+00,5.478145e+00,4.067668e+00,3.478394e+00,4.945986e+00,5.968980e+00,6.537371e+00,1.000000


## 2.2. PCA Analysis

In [39]:
## PCA Analysis

def pca_visual(df=df_ROI_for_ML_Opti_di):

    pca = PCA(n_components=3)
    raMSI_ML_mz_df = df.drop("type", axis = 1)
    pca.fit(raMSI_ML_mz_df)
    result = pd.DataFrame(pca.transform(raMSI_ML_mz_df), columns=['PCA%i' % i for i in range(3)], index=df_ROI_for_ML_Opti_di.index)
    
    plt.subplots(dpi = 300)
    plt.scatter(result['PCA1'], result['PCA2'], c = df['type'], s=1)
    plt.xlabel('PC1', size=20)
    plt.ylabel('PC2', size=20)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    plt.savefig(r"H:/3_output_raMSIn/3_3_Output_raMSIn_HKU_Ingested4ALL/XGB_ALL/afterModelSelection/dbMSIn_10ppm/PCA_di.tif", bbox_inches = 'tight')

    components = pca.fit_transform(raMSI_ML_mz_df)
    total_var = pca.explained_variance_ratio_.sum() * 100
    fig = px.scatter_3d(
        components, x=0, y=1, z=2, color=df['type'],
        title = f'Total Explained Variance: {total_var:.2f}%',
        labels = {'0': 'PC 1', '1': 'PC 2', '2': 'PC 3'}
    )
    fig.show()

In [ ]:
pca_visual()

# 3. Supervised Learning

## 3.1. Preparation of training and test dataset

In [31]:
def split_dataset(df=df_ROI_for_ML_Opti_ingested):
    raMSI_ML_NoType = df.drop("type", axis = 1)
    raMSI_ML_Type=df[['type']]
    X_train,X_test,y_train,y_test=train_test_split(raMSI_ML_NoType,raMSI_ML_Type,test_size=0.2,random_state=42)
    return X_train,X_test,y_train,y_test

X_train,X_test,y_train,y_test=split_dataset(df=df_ROI_for_ML_Opti)

In [32]:
X_ingested = df_ROI_for_ML_Opti_ingested[df_ROI_for_ML_Opti_ingested.columns[:-1]]

In [33]:
y_ingested = df_ROI_for_ML_Opti_ingested[df_ROI_for_ML_Opti_ingested.columns[-1]]

In [34]:
X_train = df_ROI_for_ML_Opti_train[df_ROI_for_ML_Opti_train.columns[:-1]]

In [35]:
y_train = df_ROI_for_ML_Opti_train[df_ROI_for_ML_Opti_train.columns[-1]]

In [36]:
X_val = df_ROI_for_ML_Opti_ext[df_ROI_for_ML_Opti_ext.columns[:-1]]

In [37]:
y_val = df_ROI_for_ML_Opti_ext[df_ROI_for_ML_Opti_ext.columns[-1]]

In [38]:
X_FNA = df_ROI_for_ML_Opti_FNA[df_ROI_for_ML_Opti_FNA.columns[:-1]]

In [39]:
y_FNA = df_ROI_for_ML_Opti_FNA[df_ROI_for_ML_Opti_FNA.columns[-1]]

In [40]:
X_di = df_ROI_for_ML_Opti_di[df_ROI_for_ML_Opti_di.columns[:-1]]

In [41]:
y_di = df_ROI_for_ML_Opti_di[df_ROI_for_ML_Opti_di.columns[-1]]

## 3.2 Machine Learning Modeling: Model 1+2 (Train 10:10, Val 4:4, Test 3:3)

In [42]:
from matplotlib.pyplot import figure
plt.rcParams['figure.figsize'] = [10, 10]
plt.grid(visible=None)
plt.close()
figure(figsize=(10, 10))

<Figure size 720x720 with 0 Axes>

### SVM

In [45]:
## Build LinearSVC Model
from sklearn.svm import LinearSVC

def LinearSVC_func(XTrain=X_ingested, XVal=X_val, XTest=X_FNA, XTest2=X_di, yTrain=y_ingested, yVal=y_val, yTest=y_FNA, yTest2=y_di):
    model_svm = LinearSVC(penalty='l2', loss="hinge", C=402, random_state=42, class_weight={0:0.9628, 1:1.0402})
    model_svm.fit(XTrain,yTrain)
    
    # Training Set
    #plt.rcParams["figure.figsize"] = (3,3)
    X_train_predictions = model_svm.predict(X_train)
    print("Training Set F1:", f1_score(y_train, model_svm.predict(X_train), sample_weight=sampleW))
    print("Training Set MCC:", matthews_corrcoef(y_train, model_svm.predict(X_train), sample_weight=sampleW))
    print("Training Set Accuracy:", metrics.accuracy_score(y_train, X_train_predictions))
    print(classification_report(y_train, X_train_predictions))
    print(confusion_matrix(y_train, X_train_predictions))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr, tpr, _ = metrics.roc_curve(y_train, X_train_predictions)
    plt.plot(fpr,tpr)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_ROC_train.tif", bbox_inches = 'tight')
    #plt.show()

    # Val set
    predictions_val  = model_svm.predict(XVal)
    print("Ext Val Set F1:", f1_score(yVal, model_svm.predict(XVal), sample_weight=sampleExtW))
    print("Ext Val Set MCC:", matthews_corrcoef(yVal, model_svm.predict(XVal), sample_weight=sampleExtW))
    print("Ext Val Set Accuracy:", metrics.accuracy_score(yVal, predictions_val))
    print(classification_report(yVal, predictions_val))
    print(confusion_matrix(yVal, predictions_val))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_val, tpr_val, _ = metrics.roc_curve(yVal, predictions_val)
    plt.plot(fpr_val,tpr_val)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_ROC_test.tif", bbox_inches = 'tight')
    #plt.show()
    
    f1_5_train = cross_val_score(model_svm, XTrain, yTrain, cv = 5, scoring="f1")
    print("5F-CV:", avgScore(f1_5_train, 5))
    
    # Test set
    predictions_FNA = model_svm.predict(XTest)
    print("FNA Set F1:", f1_score(yTest, model_svm.predict(XTest), sample_weight=sampleFNAW))
    print("FNA Set MCC:", matthews_corrcoef(yTest, model_svm.predict(XTest), sample_weight=sampleFNAW))
    print("FNA Set Recall:", recall_score(yTest, model_svm.predict(XTest)))
    print("FNA Test Set Accuracy:", metrics.accuracy_score(yTest, predictions_FNA))
    print("FNA Test Set Precision:", metrics.precision_score(yTest, predictions_FNA))
    print(classification_report(yTest, predictions_FNA))
    print(confusion_matrix(yTest, predictions_FNA))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_FNA, tpr_FNA, _ = metrics.roc_curve(yTest, predictions_FNA)
    plt.plot(fpr_FNA,tpr_FNA)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"G:/raMSIn/XGB_Importance2/XGB2_ROC_FNAtest2.tif", bbox_inches = 'tight')
    #plt.show()
    
    # Visualize the importantance of the features
    feature_imp = pd.Series(model_svm.coef_[0], index=XTrain.columns).sort_values(ascending=False)
    x=feature_imp
    y=feature_imp.index
    
    plt.subplots(dpi = 300)
    #plt.rcParams["figure.figsize"] = [4, 6]
    #plt.rcParams["font.family"] = "Arial"
    #plt.rcParams.update({'font.size': 6})
    plt.xlabel('Feature Importance Score', fontproperties='Arial', fontsize=17)
    plt.ylabel('Features', fontproperties='Arial', fontsize=17)
    plt.xticks(fontsize=15)
    plt.yticks(fontsize=15)
    plt.title("Important Molecular Features", fontproperties='Arial', fontsize=20)
    sns.barplot(x=feature_imp, y=feature_imp.index)
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_TopFeatures.tif", bbox_inches = 'tight')
    #plt.show()
    
    #feature_importances = model_svm.coef_[0]
    #feature_df=pd.DataFrame({'m/z':XTrain.columns,'coef':feature_importances})

    #fig = px.bar(feature_df, x='coef', y='m/z')
    #fig.show()
    
    # Test2 set
    predictions_di = model_svm.predict(XTest2)
    print("DirectIn Set F1:", f1_score(yTest2, model_svm.predict(XTest2), sample_weight=sampleDiW))
    print("DirectIn Set MCC:", matthews_corrcoef(yTest2, model_svm.predict(XTest2), sample_weight=sampleDiW))
    print("DirectIn Set Recall:", recall_score(yTest2, model_svm.predict(XTest2)))
    print("DirectIn Set Precision:", precision_score(yTest2, model_svm.predict(XTest2)))
    print("DirectIn Test Set Accuracy:", metrics.accuracy_score(yTest2, predictions_di))
    print(classification_report(yTest2, predictions_di))
    print(confusion_matrix(yTest2, predictions_di))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_di, tpr_di, _ = metrics.roc_curve(yTest2, predictions_di)
    plt.plot(fpr_di,tpr_di)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"G:/raMSIn/XGB_Importance2/XGB2_ROC_FNAtest2.tif", bbox_inches = 'tight')
    #plt.show()
    
    
    return x, y

In [46]:
coeff_SVM, TOPfeatures_SVM = LinearSVC_func()

Training Set F1: 0.9071822320924032
Training Set MCC: 0.8283319221490406
Training Set Accuracy: 0.914193051890941
              precision    recall  f1-score   support

           0       0.88      0.96      0.92     47449
           1       0.96      0.86      0.91     43511

    accuracy                           0.91     90960
   macro avg       0.92      0.91      0.91     90960
weighted avg       0.92      0.91      0.91     90960

[[45716  1733]
 [ 6072 37439]]
Ext Val Set F1: 0.5270206862523358
Ext Val Set MCC: 0.07858455177072926
Ext Val Set Accuracy: 0.5384362139917696
              precision    recall  f1-score   support

           0       0.52      0.57      0.54      2943
           1       0.56      0.51      0.53      3132

    accuracy                           0.54      6075
   macro avg       0.54      0.54      0.54      6075
weighted avg       0.54      0.54      0.54      6075

[[1663 1280]
 [1524 1608]]
5F-CV: 0.9073542691010594
FNA Set F1: 0.7941726073476703
FNA 

In [47]:
import joblib as jl

In [48]:
model = LinearSVC(penalty='l2', loss="hinge", C=402, random_state=42, class_weight={0:0.9628, 1:1.0402})
model.fit(X_ingested, y_ingested)

LinearSVC(C=402, class_weight={0: 0.9628, 1: 1.0402}, loss='hinge',
          random_state=42)

In [49]:
modelSavePath = r"I:\3_output_raMSIn\3_3_Output_raMSIn_HKU_Ingested4ALL\raMSIn\modeling\modelSVM_dbMSIn10ppm.joblib"
jl.dump(model, modelSavePath, compress = 5)

['I:\\3_output_raMSIn\\3_3_Output_raMSIn_HKU_Ingested4ALL\\raMSIn\\modeling\\modelSVM_dbMSIn10ppm.joblib']

## 3.3 Machine Learning Modeling: Model 1 (Train 6:6, Val 4:4, Test 3:3)

### SVM

In [50]:
## Build LinearSVC Model
from sklearn.svm import LinearSVC

def LinearSVC_func(XTrain=X_train, XVal=X_val, XTest=X_FNA, XTest2=X_di, yTrain=y_train, yVal=y_val, yTest=y_FNA, yTest2=y_di):
    model_svm = LinearSVC(penalty='l2', loss="hinge", C=402, random_state=42, class_weight={0:0.9628, 1:1.0402})
    model_svm.fit(XTrain,yTrain)
    
    # Training Set
    #plt.rcParams["figure.figsize"] = (3,3)
    X_train_predictions = model_svm.predict(X_train)
    print("Training Set F1:", f1_score(y_train, model_svm.predict(X_train), sample_weight=sampleW))
    print("Training Set MCC:", matthews_corrcoef(y_train, model_svm.predict(X_train), sample_weight=sampleW))
    print("Training Set Accuracy:", metrics.accuracy_score(y_train, X_train_predictions))
    print(classification_report(y_train, X_train_predictions))
    print(confusion_matrix(y_train, X_train_predictions))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr, tpr, _ = metrics.roc_curve(y_train, X_train_predictions)
    plt.plot(fpr,tpr)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_ROC_train.tif", bbox_inches = 'tight')
    #plt.show()

    # Val set
    predictions_val  = model_svm.predict(XVal)
    print("Ext Val Set F1:", f1_score(yVal, model_svm.predict(XVal), sample_weight=sampleExtW))
    print("Ext Val Set MCC:", matthews_corrcoef(yVal, model_svm.predict(XVal), sample_weight=sampleExtW))
    print("Ext Val Set Accuracy:", metrics.accuracy_score(yVal, predictions_val))
    print(classification_report(yVal, predictions_val))
    print(confusion_matrix(yVal, predictions_val))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_val, tpr_val, _ = metrics.roc_curve(yVal, predictions_val)
    plt.plot(fpr_val,tpr_val)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_ROC_test.tif", bbox_inches = 'tight')
    #plt.show()
    
    f1_5_train = cross_val_score(model_svm, XTrain, yTrain, cv = 5, scoring="f1")
    print("5F-CV:", avgScore(f1_5_train, 5))
    
    # Test set
    predictions_FNA = model_svm.predict(XTest)
    print("FNA Set F1:", f1_score(yTest, model_svm.predict(XTest), sample_weight=sampleFNAW))
    print("FNA Set MCC:", matthews_corrcoef(yTest, model_svm.predict(XTest), sample_weight=sampleFNAW))
    print("FNA Set Recall:", recall_score(yTest, model_svm.predict(XTest)))
    print("FNA Test Set Accuracy:", metrics.accuracy_score(yTest, predictions_FNA))
    print("FNA Test Set Precision:", metrics.precision_score(yTest, predictions_FNA))
    print(classification_report(yTest, predictions_FNA))
    print(confusion_matrix(yTest, predictions_FNA))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_FNA, tpr_FNA, _ = metrics.roc_curve(yTest, predictions_FNA)
    plt.plot(fpr_FNA,tpr_FNA)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"G:/raMSIn/XGB_Importance2/XGB2_ROC_FNAtest2.tif", bbox_inches = 'tight')
    #plt.show()
    
    # Visualize the importantance of the features
    feature_imp = pd.Series(model_svm.coef_[0], index=XTrain.columns).sort_values(ascending=False)
    x=feature_imp
    y=feature_imp.index
    
    plt.subplots(dpi = 300)
    #plt.rcParams["figure.figsize"] = [4, 6]
    #plt.rcParams["font.family"] = "Arial"
    #plt.rcParams.update({'font.size': 6})
    plt.xlabel('Feature Importance Score', fontproperties='Arial', fontsize=17)
    plt.ylabel('Features', fontproperties='Arial', fontsize=17)
    plt.xticks(fontsize=15)
    plt.yticks(fontsize=15)
    plt.title("Important Molecular Features", fontproperties='Arial', fontsize=20)
    sns.barplot(x=feature_imp, y=feature_imp.index)
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_TopFeatures.tif", bbox_inches = 'tight')
    #plt.show()
    
    #feature_importances = model_svm.coef_[0]
    #feature_df=pd.DataFrame({'m/z':XTrain.columns,'coef':feature_importances})

    #fig = px.bar(feature_df, x='coef', y='m/z')
    #fig.show()
    
    # Test2 set
    predictions_di = model_svm.predict(XTest2)
    print("DirectIn Set F1:", f1_score(yTest2, model_svm.predict(XTest2), sample_weight=sampleDiW))
    print("DirectIn Set MCC:", matthews_corrcoef(yTest2, model_svm.predict(XTest2), sample_weight=sampleDiW))
    print("DirectIn Set Recall:", recall_score(yTest2, model_svm.predict(XTest2)))
    print("DirectIn Set Precision:", precision_score(yTest2, model_svm.predict(XTest2)))
    print("DirectIn Test Set Accuracy:", metrics.accuracy_score(yTest2, predictions_di))
    print(classification_report(yTest2, predictions_di))
    print(confusion_matrix(yTest2, predictions_di))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_di, tpr_di, _ = metrics.roc_curve(yTest2, predictions_di)
    plt.plot(fpr_di,tpr_di)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"G:/raMSIn/XGB_Importance2/XGB2_ROC_FNAtest2.tif", bbox_inches = 'tight')
    #plt.show()
    
    return x, y

In [51]:
coeff_SVM, TOPfeatures_SVM = LinearSVC_func()

Training Set F1: 0.8918946248793713
Training Set MCC: 0.7867205231998081
Training Set Accuracy: 0.8937774846086192
              precision    recall  f1-score   support

           0       0.89      0.91      0.90     47449
           1       0.90      0.88      0.89     43511

    accuracy                           0.89     90960
   macro avg       0.89      0.89      0.89     90960
weighted avg       0.89      0.89      0.89     90960

[[42974  4475]
 [ 5187 38324]]
Ext Val Set F1: 0.6547340728992326
Ext Val Set MCC: 0.2598587256725162
Ext Val Set Accuracy: 0.6307818930041152
              precision    recall  f1-score   support

           0       0.64      0.55      0.59      2943
           1       0.63      0.70      0.66      3132

    accuracy                           0.63      6075
   macro avg       0.63      0.63      0.63      6075
weighted avg       0.63      0.63      0.63      6075

[[1625 1318]
 [ 925 2207]]
5F-CV: 0.7963049539702708
FNA Set F1: 0.62387809789606
FNA Se

## 3.4 Machine Learning Modeling: Model 2 (Train 4:4, Val 6:6, Test 3:3)

### SVM

In [52]:
## Build LinearSVC Model
from sklearn.svm import LinearSVC

def LinearSVC_func(XTrain=X_val, XVal=X_train, XTest=X_FNA, XTest2=X_di, yTrain=y_val, yVal=y_train, yTest=y_FNA, yTest2=y_di):
    model_svm = LinearSVC(penalty='l2', loss="hinge", C=402, random_state=42, class_weight={0:0.9628, 1:1.0402})
    model_svm.fit(XTrain,yTrain)
    
    # Training Set
    #plt.rcParams["figure.figsize"] = (3,3)
    X_train_predictions = model_svm.predict(XVal)
    print("Training Set F1:", f1_score(yVal, model_svm.predict(XVal), sample_weight=sampleW))
    print("Training Set MCC:", matthews_corrcoef(yVal, model_svm.predict(XVal), sample_weight=sampleW))
    print("Training Set Accuracy:", metrics.accuracy_score(yVal, X_train_predictions))
    print(classification_report(yVal, X_train_predictions))
    print(confusion_matrix(yVal, X_train_predictions))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr, tpr, _ = metrics.roc_curve(yVal, X_train_predictions)
    plt.plot(fpr,tpr)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_ROC_train.tif", bbox_inches = 'tight')
    #plt.show()

    # Val set
    predictions_val  = model_svm.predict(XTrain)
    print("Ext Val Set F1:", f1_score(yTrain, model_svm.predict(XTrain), sample_weight=sampleExtW))
    print("Ext Val Set MCC:", matthews_corrcoef(yTrain, model_svm.predict(XTrain), sample_weight=sampleExtW))
    print("Ext Val Set Accuracy:", metrics.accuracy_score(yTrain, predictions_val))
    print(classification_report(yTrain, predictions_val))
    print(confusion_matrix(yTrain, predictions_val))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_val, tpr_val, _ = metrics.roc_curve(yTrain, predictions_val)
    plt.plot(fpr_val,tpr_val)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_ROC_test.tif", bbox_inches = 'tight')
    #plt.show()
    
    f1_5_train = cross_val_score(model_svm, XTrain, yTrain, cv = 5, scoring="f1")
    print("5F-CV:", avgScore(f1_5_train, 5))
    
    # Test set
    predictions_FNA = model_svm.predict(XTest)
    print("FNA Set F1:", f1_score(yTest, model_svm.predict(XTest), sample_weight=sampleFNAW))
    print("FNA Set MCC:", matthews_corrcoef(yTest, model_svm.predict(XTest), sample_weight=sampleFNAW))
    print("FNA Set Recall:", recall_score(yTest, model_svm.predict(XTest)))
    print("FNA Test Set Accuracy:", metrics.accuracy_score(yTest, predictions_FNA))
    print("FNA Test Set Precision:", metrics.precision_score(yTest, predictions_FNA))
    print(classification_report(yTest, predictions_FNA))
    print(confusion_matrix(yTest, predictions_FNA))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_FNA, tpr_FNA, _ = metrics.roc_curve(yTest, predictions_FNA)
    plt.plot(fpr_FNA,tpr_FNA)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"G:/raMSIn/XGB_Importance2/XGB2_ROC_FNAtest2.tif", bbox_inches = 'tight')
    #plt.show()
    
    # Visualize the importantance of the features
    feature_imp = pd.Series(model_svm.coef_[0], index=XTrain.columns).sort_values(ascending=False)
    x=feature_imp
    y=feature_imp.index
    
    plt.subplots(dpi = 300)
    #plt.rcParams["figure.figsize"] = [4, 6]
    #plt.rcParams["font.family"] = "Arial"
    #plt.rcParams.update({'font.size': 6})
    plt.xlabel('Feature Importance Score', fontproperties='Arial', fontsize=17)
    plt.ylabel('Features', fontproperties='Arial', fontsize=17)
    plt.xticks(fontsize=15)
    plt.yticks(fontsize=15)
    plt.title("Important Molecular Features", fontproperties='Arial', fontsize=20)
    sns.barplot(x=feature_imp, y=feature_imp.index)
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_TopFeatures.tif", bbox_inches = 'tight')
    #plt.show()
    
    #feature_importances = model_svm.coef_[0]
    #feature_df=pd.DataFrame({'m/z':XTrain.columns,'coef':feature_importances})

    #fig = px.bar(feature_df, x='coef', y='m/z')
    #fig.show()
    
    # Test2 set
    predictions_di = model_svm.predict(XTest2)
    print("DirectIn Set F1:", f1_score(yTest2, model_svm.predict(XTest2), sample_weight=sampleDiW))
    print("DirectIn Set MCC:", matthews_corrcoef(yTest2, model_svm.predict(XTest2), sample_weight=sampleDiW))
    print("DirectIn Set Recall:", recall_score(yTest2, model_svm.predict(XTest2)))
    print("DirectIn Set Precision:", precision_score(yTest2, model_svm.predict(XTest2)))
    print("DirectIn Test Set Accuracy:", metrics.accuracy_score(yTest2, predictions_di))
    print(classification_report(yTest2, predictions_di))
    print(confusion_matrix(yTest2, predictions_di))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_di, tpr_di, _ = metrics.roc_curve(yTest2, predictions_di)
    plt.plot(fpr_di,tpr_di)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"G:/raMSIn/XGB_Importance2/XGB2_ROC_FNAtest2.tif", bbox_inches = 'tight')
    #plt.show()

    
    return x, y

In [53]:
coeff_SVM, TOPfeatures_SVM = LinearSVC_func()

Training Set F1: 0.7353207002259042
Training Set MCC: 0.4610645092804182
Training Set Accuracy: 0.7295624450307827
              precision    recall  f1-score   support

           0       0.76      0.71      0.73     47449
           1       0.70      0.75      0.73     43511

    accuracy                           0.73     90960
   macro avg       0.73      0.73      0.73     90960
weighted avg       0.73      0.73      0.73     90960

[[33769 13680]
 [10919 32592]]
Ext Val Set F1: 0.9017211272285746
Ext Val Set MCC: 0.8039549467205679
Ext Val Set Accuracy: 0.9018930041152263
              precision    recall  f1-score   support

           0       0.89      0.90      0.90      2943
           1       0.91      0.90      0.90      3132

    accuracy                           0.90      6075
   macro avg       0.90      0.90      0.90      6075
weighted avg       0.90      0.90      0.90      6075

[[2662  281]
 [ 315 2817]]
5F-CV: 0.8726614269221462
FNA Set F1: 0.6860337253109352
FNA 

# 4. Model Evaluation

## 4.1 Prediction

In [54]:
## deploy models for training set ##
model = jl.load(r"I:\3_output_raMSIn\3_3_Output_raMSIn_HKU_Ingested4ALL\raMSIn\modeling\modelSVM_dbMSIn10ppm.joblib")
#
## deploy model ##
predicted0n1_train = model.predict(X_train)      
trainDEFSDf["predicted0n1"] = predicted0n1_train
T_ = []
F_ = []
for i in range(len(trainDEFSDf)):
    if trainDEFSDf.iloc[i]["predicted0n1"] == 1:
        T_.append(1)
        F_.append(0)
    elif trainDEFSDf.iloc[i]["predicted0n1"] == 0:
        F_.append(1)
        T_.append(0)
trainDEFSDf["T"] = T_
trainDEFSDf["F"] = F_
#
## save ##, ouputing trainSet df
savePath = r"I:\3_output_raMSIn\3_3_Output_raMSIn_HKU_Ingested4ALL\raMSIn\modeling\df_train_dbMSIn10ppm_STDnorm_0n1.csv"
trainDEFSDf.to_csv(savePath)

In [55]:

## deploy model ##
predicted0n1_ext = model.predict(X_val)
extDEFSDf["predicted0n1"] = predicted0n1_ext
T_ = []
F_ = []
for i in range(len(extDEFSDf)):
    if extDEFSDf.iloc[i]["predicted0n1"] == 1:
        T_.append(1)
        F_.append(0)
    elif extDEFSDf.iloc[i]["predicted0n1"] == 0:
        F_.append(1)
        T_.append(0)
extDEFSDf["T"] = T_
extDEFSDf["F"] = F_
#
## save ##, ouputing extSet df
savePath = r"I:\3_output_raMSIn\3_3_Output_raMSIn_HKU_Ingested4ALL\raMSIn\modeling\df_ext_dbMSIn10ppm_STDnorm_0n1.csv"
extDEFSDf.to_csv(savePath)

In [56]:

## deploy model ##
predicted0n1_FNA = model.predict(X_FNA)
fnaDEFSDf["predicted0n1"] = predicted0n1_FNA
T_ = []
F_ = []
for i in range(len(fnaDEFSDf)):
    if fnaDEFSDf.iloc[i]["predicted0n1"] == 1:
        T_.append(1)
        F_.append(0)
    elif fnaDEFSDf.iloc[i]["predicted0n1"] == 0:
        F_.append(1)
        T_.append(0)
fnaDEFSDf["T"] = T_
fnaDEFSDf["F"] = F_
#
## save ##, ouputing extSet df
savePath = r"I:\3_output_raMSIn\3_3_Output_raMSIn_HKU_Ingested4ALL\raMSIn\modeling\df_FNA_dbMSIn10ppm_STDnorm_0n1.csv"
fnaDEFSDf.to_csv(savePath)

In [57]:

## deploy model ##
predicted0n1_Di = model.predict(X_di)
diDEFSDf["predicted0n1"] = predicted0n1_Di
T_ = []
F_ = []
for i in range(len(diDEFSDf)):
    if diDEFSDf.iloc[i]["predicted0n1"] == 1:
        T_.append(1)
        F_.append(0)
    elif diDEFSDf.iloc[i]["predicted0n1"] == 0:
        F_.append(1)
        T_.append(0)
diDEFSDf["T"] = T_
diDEFSDf["F"] = F_
#
## save ##, ouputing extSet df
savePath = r"I:\3_output_raMSIn\3_3_Output_raMSIn_HKU_Ingested4ALL\raMSIn\modeling\df_Di_dbMSIn10ppm_STDnorm_0n1.csv"
diDEFSDf.to_csv(savePath)